# Creating a SKOS Thesaurus from the cleaned signature schema

We'll use [RDFlib](https://rdflib.readthedocs.io) for managing SKOS terms etc. You don't _have_ to as it is quite possible to just output RDF through string manipulation, but seeing as this is not a huge dataset, we can afford to go through an in-memory RDF graph.

In [ ]:
import pandas as pd
import re
from rdflib import Graph, Namespace, RDF
from rdflib.namespace import SKOS

## Load the data
We assume to be working with the final output of the [signatures_processing](signatures_processing.ipynb) notebook.

In [2]:
df = pd.read_csv('data/csv/sig_updated.csv',dtype={'numbis': str, 'backreference': str, 'text_4': str})

# Create a multi-index as we might need to access rows over and over.

df.set_index(['lev','sys','numbis'], inplace=True)
df.sort_index()

text   
lev sys     numbis                                                      
1.0 A       NaN                                          Handbücherei  \
    B       NaN                                    Italienische Kunst   
    D       NaN                                       Topographie Rom   
    F       NaN                                         Reiseberichte   
    G       NaN                     Quellenschriften und Quellenkunde   
...                                                               ...   
6.0 Hh 6946 NaN     19. und 20. Jh. international, sonstige einzel...   
    Hh 6960 NaN                                         Deesis-Bilder   
    Hh 6990 NaN                                 sonstige Einzelfragen   
NaN NaN     NaN                                 Bibliotheca Hertziana   
            NaN                        Systematischer Standortkatalog   

                                    backreference        text_1   
lev sys     numbis                                                
1.0 A       NaN                               NaN           NaN  \
    B       NaN                               NaN           NaN   
    D       NaN                               NaN           NaN   
    F       NaN                               NaN           NaN   
    G       NaN                               NaN           NaN   
...                                           ...           ...   
6.0 Hh 6946 NaN     19. und 20. Jh. international  Ikonographie   
    Hh 6960 NaN                               NaN  Ikonographie   
    Hh 6990 NaN                               NaN  Ikonographie   
NaN NaN     NaN                               NaN           NaN   
            NaN                               NaN           NaN   

                                               text_2   
lev sys     numbis                                      
1.0 A       NaN                                   NaN  \
    B       NaN                                   NaN   
    D       NaN                                   NaN   
    F       NaN                                   NaN   
    G       NaN                                   NaN   
...                                               ...   
6.0 Hh 6946 NaN     Biblisch-christlicher Themenkreis   
    Hh 6960 NaN     Biblisch-christlicher Themenkreis   
    Hh 6990 NaN     Biblisch-christlicher Themenkreis   
NaN NaN     NaN                                   NaN   
            NaN                                   NaN   

                                                               text_3   
lev sys     numbis                                                      
1.0 A       NaN                                                   NaN  \
    B       NaN                                                   NaN   
    D       NaN                                                   NaN   
    F       NaN                                                   NaN   
    G       NaN                                                   NaN   
...                                                               ...   
6.0 Hh 6946 NaN     Der kanonische Bilderkreis (Altes und Neues Te...   
    Hh 6960 NaN     Der kanonische Bilderkreis (Altes und Neues Te...   
    Hh 6990 NaN     Der kanonische Bilderkreis (Altes und Neues Te...   
NaN NaN     NaN                                                   NaN   
            NaN                                                   NaN   

                                        text_4  
lev sys     numbis                              
1.0 A       NaN                            NaN  
    B       NaN                            NaN  
    D       NaN                            NaN  
    F       NaN                            NaN  
    G       NaN                            NaN  
...                                        ...  
6.0 Hh 6946 NaN     Apokalypse und Weltgericht  
    Hh 6960 NaN     Apokalypse und Weltgericht  
    Hh 6990 NaN     Apokalypse und Weltgericht  
NaN NaN     

In [ ]:
g = Graph()
NS_DATA = Namespace('http://data.biblhertz.it/term/sys/')

for index, row in df.iterrows():
    try:
        sys_uri = re.sub(r'\s+', '/', str(index[1]).strip())
        g.add((NS_DATA[sys_uri], RDF.type, SKOS.Concept))
    except KeyError:
        # Better to ask for forgiveness than for permission
        pass
    

In [ ]:
len(g)